In [397]:
import os, sys, warnings
from pathlib import Path
from IPython.display import Image, display
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
#sys.path.insert(0, os.path.abspath(os.path.join(Path.home(), "rpa_libs")))

from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys

# Nossas libs
#import app
#from plugins.selenium.selenium import Selenium

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [210]:
# Exemplo de uso do arquivo de configuração
#app.redefine_config('caminho do seu arquivo .yaml')
#teste
import pandas as pd

In [211]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [212]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install().replace("THIRD_PARTY_NOTICES.", "")))

In [213]:
url = 'https://www.coingecko.com/pt'

In [214]:
driver.get(url)

In [215]:
table = driver.find_element(By.XPATH, '/html/body/div[2]/main/div/div[5]/table')

In [216]:
rows = table.find_elements(By.XPATH, './/tbody//tr')

In [402]:
table_data = []

In [404]:
for row in rows:
    cells = row.find_elements(By.XPATH, './/td')
    cell_data = []

    for cell in cells:
        img_tag = cell.find_elements(By.TAG_NAME, 'img')
        if img_tag:
            # Extract the image URL (src attribute)
            img_src = img_tag[0].get_attribute('src')
            cell_data.append(img_src)
        
        else if :
            cell_data.append(cell.text)

    table_data.append(cell_data)

In [425]:
headers = [header.text for header in table.find_elements(By.XPATH, './/thead//th')]
headers = [item for item in headers if item]
headers[0] = "IMG Coin"
headers.insert(0, "ID")
#table_data = [[cell for cell in row if cell.strip()] for row in table_data]

In [426]:
list(headers)

['ID',
 'IMG Coin',
 'Moeda',
 'Preço',
 '1 h',
 '24 h',
 '7 d',
 'Volume em 24 h',
 'Capitalização de mercado',
 'Últimos 7 dias']

In [427]:
list(table_data)

[['',
  '1',
  'https://assets.coingecko.com/coins/images/1/standard/bitcoin.png?1696501400',
  'Comprar',
  'US$ 63.510,78',
  '1.2%',
  '0.1%',
  '6.3%',
  '',
  'US$ 27.908.976.296',
  'US$ 1.242.093.117.563',
  '',
  '',
  'https://www.coingecko.com/coins/1/sparkline.svg'],
 ['',
  '2',
  'https://assets.coingecko.com/coins/images/279/standard/ethereum.png?1696501628',
  'Comprar',
  'US$ 2.615,07',
  '1.6%',
  '0.9%',
  '12.1%',
  '',
  'US$ 16.777.671.908',
  'US$ 312.644.440.924',
  '',
  '',
  'https://www.coingecko.com/coins/279/sparkline.svg'],
 ['',
  '3',
  'https://assets.coingecko.com/coins/images/325/standard/Tether.png?1696501661',
  'Comprar',
  'US$ 0,999',
  '0.2%',
  '0.1%',
  '0.3%',
  '',
  'US$ 44.537.543.406',
  'US$ 119.188.215.909',
  '',
  '',
  'https://www.coingecko.com/coins/325/sparkline.svg'],
 ['',
  '4',
  'https://assets.coingecko.com/coins/images/825/standard/bnb-icon2_2x.png?1696501970',
  'Comprar',
  'US$ 592,36',
  '1.1%',
  '2.2%',
  '10.7%',
  

In [428]:
df = pd.DataFrame(table_data)
df_treated = df.loc[:, (df != '').any(axis=0)]
df_treated = df_treated.drop(df.columns[3], axis=1)


In [385]:
#df.style
#df.style.highlight_max()
#df.style.highlight_null(null_color="green")
#df.style.set_na_rep("OutofScope").highlight_null(null_color="orange")
#df.style.background_gradient()
#df.style.set_properties(**{'border': '1.3px solid black',
 #                         'color': 'white'})
#df.style.hide_index()

In [429]:
 df_treated = df_treated.reset_index(drop=True)
#df_treated.style\
#    .highlight_max(subset=[4], color='green')\
#    .background_gradient(cmap='viridis', subset=[4])\
#    .set_properties(**{'border': '1.3px solid black', 'color': 'white'})
#    

In [430]:
df_treated

,1,2,4,5,6,7,9,10,13
0,1,https://assets.coingecko.com/coins/images/1/st...,"US$ 63.510,78",1.2%,0.1%,6.3%,US$ 27.908.976.296,US$ 1.242.093.117.563,https://www.coingecko.com/coins/1/sparkline.svg
1,2,https://assets.coingecko.com/coins/images/279/...,"US$ 2.615,07",1.6%,0.9%,12.1%,US$ 16.777.671.908,US$ 312.644.440.924,https://www.coingecko.com/coins/279/sparkline.svg
2,3,https://assets.coingecko.com/coins/images/325/...,"US$ 0,999",0.2%,0.1%,0.3%,US$ 44.537.543.406,US$ 119.188.215.909,https://www.coingecko.com/coins/325/sparkline.svg
3,4,https://assets.coingecko.com/coins/images/825/...,"US$ 592,36",1.1%,2.2%,10.7%,US$ 1.267.576.252,US$ 87.739.917.540,https://www.coingecko.com/coins/825/sparkline.svg
4,5,https://assets.coingecko.com/coins/images/4128...,"US$ 149,85",2.0%,1.6%,10.0%,US$ 3.034.573.404,US$ 67.854.488.352,https://www.coingecko.com/coins/4128/sparkline...
...,...,...,...,...,...,...,...,...,...
95,96,https://assets.coingecko.com/coins/images/9672...,"US$ 0,1361",1.7%,0.2%,4.1%,US$ 12.356.853,US$ 777.833.222,https://www.coingecko.com/coins/9672/sparkline...
96,97,https://assets.coingecko.com/coins/images/3345...,"US$ 0,008262",2.5%,7.1%,2.5%,US$ 156.842.253,US$ 774.276.103,https://www.coingecko.com/coins/33453/sparklin...
97,98,https://assets.coingecko.com/coins/images/1233...,"US$ 28,97",3.0%,0.7%,8.0%,US$ 21.527.722,US$ 769.219.667,https://www.coingecko.com/coins/12335/sparklin...
98,99,https://assets.coingecko.com/coins/images/1302...,"US$ 5,15",2.2%,0.3%,7.7%,US$ 33.556.858,US$ 758.308.860,https://www.coingecko.com/coins/13029/sparklin...


In [431]:
df_treated.columns = [headers]
df_treated

ValueError: Length mismatch: Expected axis has 9 elements, new values have 10 elements

In [ ]:
for row in rows:
    cells = row.find_elements(By.XPATH, './/td')
    table_data.append([cell.text for cell in cells])

In [ ]:
for row in rows:
    cells = row.find_elements(By.XPATH, './/td')
    table_data.append([cell.text for cell in cells])